# Initialise GEE

In [1]:
import os
import ee
import geemap
import json
import requests

from PIL import Image

In [4]:
# Initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine already initialized")
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    print("Earth Engine initialized")

Earth Engine already initialized


## Plot the basemap

In [2]:
Map = geemap.Map(lite_mode=True)
Map.add_basemap("SATELLITE")
Map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

## Import AOI

In [4]:
# Option 1: Read JSON from file
def load_aoi_from_file(json_file_path):
    with open(json_file_path, 'r') as f:
        geojson = json.load(f)
    
    # Create an ee.Geometry from the GeoJSON
    return ee.Geometry(geojson)

In [5]:
# Option 1: Define AOI as a geometry
# aoi = ee.Geometry.Rectangle([-122.5, 37.5, -122.0, 38.0])  # San Francisco area

aoi_name = 'example'
aoi = load_aoi_from_file(fr'C:\Users\for329\OneDrive - Queensland University of Technology\FirstByte Waterholes WD\counting_waterholes\data\polygons\{aoi_name}.geojson')

# Show the AOI on map
Map.addLayer(aoi, {}, 'Area of Interest')
# Center the map on the AOI and zoom to it
Map.centerObject(aoi, zoom=10)  # Adjust zoom level (1-20) as needed
Map  # Display the map with the AOI

Map(bottom=812.0, center=[-13.055422720179042, 134.62864547283795], controls=(ZoomControl(options=['position',…

# Get Sentinel-2 Collection with Cloud Masking
This function creates a cloud mask for Sentinel-2 imagery:

In [8]:
def maskS2clouds_SCL(image):
    """
    Function to mask clouds using the SCL band from Sentinel-2 L2A data.
    SCL values:
    1: Saturated or defective
    2: Dark Area Pixels 
    3: Cloud Shadows
    4: Vegetation
    5: Bare Soils
    6: Water
    7: Clouds Low Probability / Unclassified
    8: Clouds Medium Probability
    9: Clouds High Probability
    10: Cirrus
    11: Snow / Ice
    """
    # Get the Scene Classification Layer
    scl = image.select('SCL')

    # Create a water mask to keep water bodies (value 6 is water)
    waterMask = scl.eq(6)
    
    # Keep pixels that are not clouds (values 7, 8, 9, 10) and not shadows (3)
    # Also exclude saturated/defective pixels (1)
    # Values to mask: 1, 3, 7, 8, 9, 10
    cloudShadowMask  = scl.neq(1).And(scl.neq(3)).And(
        scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)) #.And(scl.neq(7)) # removed waterbodies
    
    # Combine masks: keep pixels that are either water OR pass the cloud/shadow mask
    finalMask = waterMask.Or(cloudShadowMask)
    
    # Scale the image to 0-1 range (default is 0-10000 for SR data)
    scaled = image.divide(10000)
    
    # Apply the mask
    return scaled.updateMask(finalMask).copyProperties(image, ['system:time_start'])

## Get Monthly Composites of Sentinel-2

In [10]:
def get_monthly_composites(start_date, end_date, aoi):
    """Generate monthly S2 composites for the given date range and AOI."""
    # Convert date strings to ee.Date objects
    start = ee.Date(start_date)
    end = ee.Date(end_date)
    
    # Get number of months
    months = end.difference(start, 'month').round().int()

    # First, check if we have any S2 data for this month (without filtering)
    raw_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date)
    
    raw_count = raw_collection.size().getInfo()
    print(f"  Found {raw_count} raw S2 images for {start_date} to {end_date}")
    
    # Function to get image for a specific month
    def get_monthly_image(month_index):
        # Calculate current month start and end
        current_month_start = start.advance(month_index, 'month')
        current_month_end = current_month_start.advance(1, 'month')
        
        # Format dates for naming
        date_format = current_month_start.format('YYYY-MM')
        
        # Get S2 collection for this month
        s2_collection = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterBounds(aoi) \
            .filterDate(current_month_start, current_month_end) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 90)) \
            .map(maskS2clouds_SCL)
        
        # # Check if we have any images left after cloud filtering
        # filtered_count = s2_collection.size().getInfo()
        # print(f"  After cloud filtering: {filtered_count} images remain")
        
        # Create a median composite and clip to AOI
        composite = s2_collection.median().clip(aoi)
        
        # Select RGB bands and scale for visualization
        # Bands: B2 (blue), B3 (green), B4 (red)
        # These get re-ordered to RGB by the image_cutting_support.create_padded_png_S2 function
        rgb = composite.select(['B2', 'B3', 'B4'])
        
        return rgb.set({
            'system:index': date_format,
            'system:time_start': current_month_start.millis()
        })
    
    # Create a list of months
    month_indices = ee.List.sequence(0, months)
    
    # Map the function over the months
    composites = ee.ImageCollection.fromImages(
        month_indices.map(get_monthly_image)
    )
    
    return composites

# Export and Download Images

In [ ]:
def export_monthly_images(composites, folder_path, scale=10):
    """Export monthly images to local files."""
    # Create folder if it doesn't exist
    os.makedirs(folder_path, exist_ok=True)
    
    # Get the list of images
    image_list = composites.toList(composites.size())
    num_images = image_list.size().getInfo()
    
    # Loop through each image and export
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        date_str = image.get('system:index').getInfo()
        
        print(f"Processing image {i+1}/{num_images} for date {date_str}...")
        
        # Define output path
        geotiff_path = os.path.join(folder_path, f'{aoi_name}_{date_str}.tif')
        
        # Get download URL for the GeoTIFF
        download_url = image.getDownloadURL({
            'scale': scale,
            'crs': 'EPSG:4326',
            'fileFormat': 'GeoTIFF',
            'region': image.geometry()
        })
        
        # Download the file
        response = requests.get(download_url, stream=True)
        response.raise_for_status()  # Raise an exception for HTTP errors
        
        # Save the file
        with open(geotiff_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024*1024):
                f.write(chunk)
        
        print(f"Saved GeoTIFF: {geotiff_path}")

# For larger or higher resolution images, use Google Drive export:
def export_to_drive(composites, drive_folder='Earth_Engine_Exports', aoi_name='S2_', scale=10):
    """Export images to Google Drive for larger areas or higher resolution."""
    # Get the list of images
    image_list = composites.toList(composites.size())
    num_images = image_list.size().getInfo()
    
    # Loop through each image and export
    for i in range(num_images):
        image = ee.Image(image_list.get(i))
        date_str = image.get('system:index').getInfo()
        
        # Create export task
        task = ee.batch.Export.image.toDrive(
            image=image.visualize(min=0, max=0.3, bands=['B4', 'B3', 'B2']),
            description=f'{date_str}_{aoi_name}_S2',
            folder=drive_folder,
            scale=scale,
            region=aoi.getInfo()['coordinates'],
            maxPixels=1e9
        )
        
        # Start the task
        task.start()
        print(f"Started export task for {date_str}")

# Call functions

In [12]:
# Define parameters
start_date = '2024-01-01'
end_date = '2024-12-31' # the function will 
output_folder = 'sentinel2_monthly_images'

In [13]:
# Get monthly composites
composites = get_monthly_composites(start_date, end_date, aoi)

  Found 73 raw S2 images for 2024-01-01 to 2024-12-31


In [14]:
# Visualize a preview
map2 = geemap.Map()
map2.centerObject(aoi, 10)
map2.addLayer(aoi, {}, 'Area of Interest')

# Add the first image to the map
first_image = ee.Image(composites.first())
viz_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
# map2.addLayer(first_image, viz_params, 'First Monthly Composite')
# map2

# Get the number of images in the collection
count = composites.size().getInfo()
print(f"Found {count} monthly composites")

# Add each monthly composite to the map with its date as the layer name
if count > 0:
    # Convert to list for iterating
    image_list = composites.toList(count)
    
    # Add each image as a separate layer
    for i in range(count):
        image = ee.Image(image_list.get(i))
        date_str = image.get('system:index').getInfo()  # or 'date' if you set that property
        
        # Add to map - only the first layer will be visible by default
        map2.addLayer(image, viz_params, f'S2 RGB {date_str}', shown=(i==0))
        
    print(f"Added {count} layers to the map. Use the layer control to toggle visibility.")
else:
    print("No composites available to display.")

# Add the layer control (if not already visible)
map2.add_layer_control()

# Display the map
map2

Found 13 monthly composites
Added 13 layers to the map. Use the layer control to toggle visibility.


Map(center=[-13.055422720179042, 134.62864547283795], controls=(WidgetControl(options=['position', 'transparen…

In [15]:
print(f'AOI name:   {aoi_name}')

# Export images locally (for small areas)
# export_monthly_images(composites, output_folder)

# Or export to Google Drive (for larger areas)
export_to_drive(composites, drive_folder='sentinel2_images', aoi_name=aoi_name, scale=10)

AOI name:   example
Started export task for 2024-01
Started export task for 2024-02
Started export task for 2024-03
Started export task for 2024-04
Started export task for 2024-05
Started export task for 2024-06
Started export task for 2024-07
Started export task for 2024-08
Started export task for 2024-09
Started export task for 2024-10
Started export task for 2024-11
Started export task for 2024-12
Started export task for 2025-01
